## Preparació

In [1]:
# Paquets

import pandas as pd
import geopandas as gpd
import zipfile
import urllib.request
import os

In [2]:
# Paràmetres

# Directori de treball
WORK_DIR = os.path.abspath(os.getcwd())

# Directoris
if not os.path.exists('downloads'):
    os.makedirs('downloads')
if not os.path.exists('tmp'):
    os.makedirs('tmp')
if not os.path.exists('map'):
    os.makedirs('map')
if not os.path.exists('datasets'):
    os.makedirs('datasets')

# Enllaços de descàrrega
URL_RONDA_VERDA = 'https://opendata-ajuntament.barcelona.cat/data/dataset/18990935-6b67-49d4-9e92-2171d439ee59/resource/9c0a6e3b-ff5b-43ae-83a5-4219ad7e5e12/download'
URL_CARRIL_BICI = 'https://opendata-ajuntament.barcelona.cat/data/dataset/e3497ea4-0bae-4093-94a7-119df50a8a74/resource/357d14d1-c312-450d-a900-c15c3160a187/download'
URL_CARRIL_BICI_ALTRES_MUNICIPIS = 'https://opendata-ajuntament.barcelona.cat/data/dataset/bcbfe860-7838-468c-b78e-f15d3b9e01f8/resource/b1652bb0-33c1-4c66-81a6-ad46c8504e7e/download'
URL_CARRIL_BICI_CONSTRUCCIO = 'https://opendata-ajuntament.barcelona.cat/data/dataset/edee68e3-5ff1-43d0-b556-12c5d8eb019e/resource/3535ae40-e365-42be-adcf-3a5301948e0b/download'
URL_VIES_CICLABLES = 'https://opendata-ajuntament.barcelona.cat/data/dataset/ae147deb-1b95-4b25-aa3a-b57067f000bc/resource/af4287f7-d1d6-4988-95b0-bf00ff8b31b0/download'
URL_ZONES30_CARRERS = 'https://opendata-ajuntament.barcelona.cat/data/dataset/2258826a-67ba-4429-a500-30472ff6251a/resource/4ae49ce7-df4f-4d76-90e5-bd0e08ba1e4e/download'
URL_CARRERS_PACIFICATS = 'https://opendata-ajuntament.barcelona.cat/data/dataset/335bceee-7988-4899-b2fa-accba6a917c7/resource/d9f9d46c-d55c-419c-93f6-02b3d26beb00/download'
URL_CORREDORS_BICI = 'https://opendata-ajuntament.barcelona.cat/data/dataset/af083278-6873-4a3c-b3d5-68860cc769af/resource/4ee104ac-945e-44c8-8459-b499c9113340/download'


In [3]:
# Funcions

def get_data(url, filename):
    """Descarrega i descomprimeix les dades a la carpeta 'datasets'."""
    try:
        urllib.request.urlretrieve(url, WORK_DIR + '/downloads/' + filename + '.zip')
        zf = zipfile.ZipFile('downloads/' + filename + ".zip")
        zf.extractall(path = 'datasets/' + filename)
        print(filename + " descarregat correctament.")
    except:
        print("Problema descarregant: " + filename)
        
def save_data(gdf, filename):
    """Guarda un geodataframe en format SHP, el comprimeix i el desa a la carpeta 'map'."""
    gdf.to_file('tmp/' + filename + '.shp')
    zf = zipfile.ZipFile('map/' + filename + ".zip", "w", zipfile.ZIP_DEFLATED)
    zf.write('tmp/' + filename + ".dbf")
    zf.write('tmp/' + filename + ".shp")
    zf.write('tmp/' + filename + ".prj")
    zf.write('tmp/' + filename + ".shx")
    zf.write('tmp/' + filename + ".cpg")
    zf.close()
    print(filename + ' guardat correctament.')

In [4]:
# Captura de dades

get_data(URL_CARRIL_BICI, 'CARRIL_BICI')
get_data(URL_CARRIL_BICI_ALTRES_MUNICIPIS, 'CARRIL_BICI_ALTRES_MUNICIPIS')
get_data(URL_CARRIL_BICI_CONSTRUCCIO, 'CARRIL_BICI_CONSTRUCCIO')
get_data(URL_RONDA_VERDA, 'RONDA_VERDA')
get_data(URL_VIES_CICLABLES, 'VIES_CICLABLES')
get_data(URL_ZONES30_CARRERS, 'ZONES30_CARRERS')
get_data(URL_CARRERS_PACIFICATS, 'CARRERS_PACIFICATS')
get_data(URL_CORREDORS_BICI, 'CORREDORS_BICI')

CARRIL_BICI descarregat correctament.
CARRIL_BICI_ALTRES_MUNICIPIS descarregat correctament.
CARRIL_BICI_CONSTRUCCIO descarregat correctament.
RONDA_VERDA descarregat correctament.
VIES_CICLABLES descarregat correctament.
ZONES30_CARRERS descarregat correctament.
CARRERS_PACIFICATS descarregat correctament.
CORREDORS_BICI descarregat correctament.


## Carrils bici

* Bidireccional en vorera - Permet la circulació de bicicletes en tots dos sentits de la circulació.  Com a exemples d’aquest tipus de carril, tenim l’avinguda Diagonal, des de Francesc Macià al Palau de Pedralbes, o la Gran Via de les Corts Catalanes des de plaça d'Espanya a l'avinguda del Carrilet.
* Unidireccional en vorera - Permet la circulació en un únic sentit. És el que trobem a les tercianes de la Gran Via de les Corts Catalanes, entre la plaça d’Espanya i el carrer d’Aribau, o entre els carrers de Pau Claris i la Marina; i a les tercianes de l'avinguda Diagonal, entre el passeig de Gràcia i Sicília.
* Bidireccional en calçada - Permet la circulació de bicicletes en els dos sentits de la marxa. Normalment se situen a l'esquerra de la via en el sentit de la circulació dels altres vehicles (carrers de Calàbria, Marquès de Sentmenat, Urgell, Sicília, Provença, Teodor Llorente, etcètera). D'aquesta manera, el ciclista que circula en sentit contrari al trànsit rodat va més protegit. Aquest tipus de carril també pot estar situat al mig de la via, com per exemple al passeig de Sant Joan o a l’avinguda del Paral·lel.
* Unidireccional en calçada - Pot ser en el mateix sentit de la circulació (com les de l’avinguda Diagonal, la plaça de Francesc Macià, el passeig de Gràcia o els carrers Diputació i el Consell de Cent), o en sentit contrari, emprat especialment en carrers bici (30) per permetre la circulació de bicicletes en sentit contrari al propi de la via. És el cas de la carretera de la Bordeta o el carrer de la Jota.

In [43]:
# Carril bici

carril_bici = gpd.read_file('datasets/CARRIL_BICI/CARRIL_BICI.shp')
carril_bici['longitud_km'] = carril_bici.length / 1000
carril_bici['TOOLTIP'] = carril_bici['TOOLTIP'].apply(lambda x: x[12:].split(' ',1))
carril_bici['carrer'] = carril_bici['TOOLTIP'].apply(lambda x: x[1])
carril_bici['sentit'] = carril_bici['TOOLTIP'].apply(lambda x: x[0])
carril_bici.drop(columns=['CODI_CAPA','CODI_SUBCA','TOOLTIP'], inplace=True)
carril_bici.head(2)

,ID,geometry,longitud_km,carrer,sentit
0,GL387150,"LINESTRING (432734.577 4586875.597, 432560.232...",0.178044,Carrer Mossen Quintí Mallofré,unidireccional
1,GL387151,"LINESTRING (429423.506 4583032.909, 429393.981...",0.688776,Via Augusta lateral Besòs,unidireccional


In [44]:
# Carril bici altres municipis

carril_bici_altres = gpd.read_file('datasets/CARRIL_BICI_ALTRES_MUNICIPIS/CARRIL_BICI_ALTRES_MUNICIPIS.shp')
carril_bici_altres['longitud_km'] = carril_bici_altres.length / 1000
carril_bici_altres['municipi'] = carril_bici_altres['TOOLTIP'].apply(lambda x: x[29:])
carril_bici_altres.drop(columns=['CODI_CAPA','CODI_SUBCA', 'TOOLTIP'], inplace=True)
carril_bici_altres.head(2)

,ID,geometry,longitud_km,municipi
0,GL385847,"LINESTRING (425738.483 4575492.044, 425716.295...",3.169390,El Prat 1
1,GL385817,"LINESTRING (435172.426 4585026.889, 434294.111...",2.003901,Sant Ramón de Penyafort


In [45]:
# Carril bici en construcció

carril_bici_construccio = gpd.read_file('datasets/CARRIL_BICI_CONSTRUCCIO/CARRIL_BICI_CONSTRUCCIO.shp')
carril_bici_construccio['longitud_km'] = carril_bici_construccio.length / 1000
carril_bici_construccio['carrer'] = carril_bici_construccio['TOOLTIP'].apply(lambda x: x[27:])
carril_bici_construccio.drop(columns=['CODI_CAPA','CODI_SUBCA', 'TOOLTIP'], inplace=True)
carril_bici_construccio.head(2)

,ID,geometry,longitud_km,carrer
0,GL411365,"LINESTRING (426639.380 4582064.042, 426622.491...",0.779218,Bidireccional av Pius XII
1,GL412073,"LINESTRING (428536.603 4578089.355, 427737.746...",1.704288,pg Zona Franca - sentit muntanya


## Vies ciclables

* Zones de vianants amb plataforma única (carrer 10 i 20) - Aquestes zones de prioritat invertida són àrees o carrers on les condicions de circulació de vehicles queden restringides a favor de la circulació de vianants. En aquestes zones, les bicicletes, els patins i els patinets gaudeixen de prioritat sobre la resta de vehicles, però no sobre els vianants, i poden circular en tots dos sentits de la via.
* Carrer Bici (carrer 30) - Són carrers de trànsit pacificat en els quals es promou la circulació de ciclistes, ja que el tràfic d'automòbils té limitada la seva velocitat a 30 km/h. En aquests carrers, les bicicletes han de circular en el sentit de la circulació de la via i tenen prioritat sobre la circulació de la resta de vehicles, però no sobre els vianants.


In [46]:
# Vies ciclables

vies_ciclables = gpd.read_file('datasets/VIES_CICLABLES/VIES_CICLABLES.shp')
vies_ciclables = vies_ciclables.to_crs(carril_bici.crs)
vies_ciclables['longitud_km'] = vies_ciclables.length / 1000
vies_ciclables['carrer'] = vies_ciclables['TOOLTIP'].apply(lambda x: x[14:])
vies_ciclables.drop(columns=['CODI_CAPA','CODI_SUBCA','TOOLTIP'], inplace=True)
vies_ciclables.head(2)

,ID,geometry,longitud_km,carrer
0,GL387131,"LINESTRING (428457.967 4578985.824, 428414.724...",0.848331,Carrer ferrocarrils Catalans
1,GL387132,"LINESTRING (428673.207 4578158.057, 428668.906...",0.390423,Carrer Encuny


## Zona 30

In [47]:
zones30 = gpd.read_file('datasets/ZONES30_CARRERS/ZONES30_CARRERS.shp')
zones30['longitud_km'] = zones30.length / 1000
zones30.drop(columns=['CODI_CAPA','CODI_SUBCA','TOOLTIP'], inplace=True)
zones30.head(2)

,ID,geometry,longitud_km
0,GL77995,"LINESTRING (428935.851 4583245.483, 428874.674...",0.076005
1,GL77996,"LINESTRING (428929.413 4583329.827, 428887.076...",0.064934


## Ronda verda

In [52]:
# ronda_verda verda

ronda_verda = gpd.read_file('datasets/RONDA_VERDA/RONDA_VERDA.shp')
ronda_verda['longitud_km'] = ronda_verda.length / 1000
ronda_verda['carrer'] = ronda_verda['TOOLTIP'].apply(lambda x: x.split(': ',1)[1])
ronda_verda.drop(columns=['CODI_CAPA','CODI_SUBCA', 'TOOLTIP'], inplace=True)
ronda_verda.head(2)

,ID,geometry,longitud_km,carrer
0,GL385629,"LINESTRING (430754.622 4589414.253, 430784.587...",6.305380,ronda verda 2
1,GL385569,"LINESTRING (431077.126 4580518.898, 431094.584...",4.058222,Ronda Colom - Moll Barceloneta - Pg. Marítim d...


## Corredors bici

In [49]:
# Corredors bici

corredors_bici = gpd.read_file('datasets/CORREDORS_BICI/CORREDORS_BICI.shp')
corredors_bici = corredors_bici.to_crs(carril_bici.crs)
corredors_bici['longitud_km'] = corredors_bici.length / 1000
corredors_bici.drop(columns=['CODI_CAPA','CODI_SUBCA','TOOLTIP'], inplace=True)
corredors_bici.head(2)

,ID,geometry,longitud_km
0,GL411915,"LINESTRING (428780.552 4580888.193, 428738.287...",0.159959
1,GL411916,"LINESTRING (428871.119 4580974.447, 429078.991...",0.320414


## Carrers pacificats

In [50]:
# Carrers pacificats

carrers_pacificats = gpd.read_file('datasets/CARRERS_PACIFICATS/CARRERS_PACIFICATS.shp')
carrers_pacificats = carrers_pacificats.to_crs(carril_bici.crs)
carrers_pacificats['longitud_km'] = carrers_pacificats.length / 1000
carrers_pacificats.drop(columns=['CODI_CAPA','CODI_SUBCA','TOOLTIP'], inplace=True)
carrers_pacificats.head(2)

,ID,geometry,longitud_km
0,GL386984,"LINESTRING (428886.421 4581537.262, 429275.555...",0.561546
1,GL386985,"LINESTRING (429067.231 4581348.590, 430690.589...",2.273649


## Exportació de dades

In [53]:
save_data(carril_bici, 'CARRIL_BICI')
save_data(vies_ciclables, 'VIES_CICLABLES')
save_data(zones30, 'ZONES30')
save_data(ronda_verda, 'RONDA_VERDA')
save_data(corredors_bici, 'CORREDORS_BICI')
save_data(carrers_pacificats, 'CARRERS_PACIFICATS')
save_data(carril_bici_altres, 'CARRIL_BICI_ALTRES_MUNICIPIS')
save_data(carril_bici_construccio, 'CARRIL_BICI_CONSTRUCCIO')

CARRIL_BICI guardat correctament.
VIES_CICLABLES guardat correctament.
ZONES30 guardat correctament.
RONDA_VERDA guardat correctament.
CORREDORS_BICI guardat correctament.
CARRERS_PACIFICATS guardat correctament.
CARRIL_BICI_ALTRES_MUNICIPIS guardat correctament.
CARRIL_BICI_CONSTRUCCIO guardat correctament.
